# 1. Get an NLI pipline by huggingface

In [ ]:
!pip install transformers
!pip install captum

     |████████████████████████████████| 2.9 MB 8.0 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 58.7 MB/s 
     |████████████████████████████████| 895 kB 61.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.4 MB 7.7 MB/s 


In [ ]:
import torch
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [ ]:
import transformers
from transformers import pipeline
NLIer = pipeline("text-classification", model='/content/drive/MyDrive/outputs/fewshotdeberta')

In [ ]:
def loadData(source_path, target_path):
    custom_dataset = []
    with open(source_path) as F:
        with open(target_path) as G:
            f=F.read().split('\n')
            g=G.read().split('\n')
            for i in range(len(f)):
                if ('2' in g[i]):
                    custom_dataset+=[(f[i], 2)]
    return custom_dataset

custom_dataset = loadData("/content/drive/MyDrive/outputs/test_sentence.source", "/content/drive/MyDrive/outputs/test_sentence.target")

In [ ]:
def preprocess_data(tokenizer, sentense):
    return tokenized_sentense

In [ ]:
NLIer(custom_dataset[0][0])

[{'label': 'LABEL_2', 'score': 0.9970428347587585}]

# 2. Generate Latex format

In [ ]:
model, tokenlizer = None, None
import torch
loss=torch.nn.CrossEntropyLoss()

def squad_pos_forward_func(inputs, position):
    pred = torch.nn.functional.softmax(model(inputs).logits)
    label = torch.tensor([[position]*pred.shape[0]]).cuda()
    return loss(pred, torch.tensor([position]*pred.shape[0]).cuda()).view([1])

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions[0] = 0
    attributions[-1] = 0
    attributions = attributions / torch.max(torch.abs(attributions))

    return [float(i)*100 for i in attributions]

def generate_HeatmapText(attention_list, text_list):
    string = r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
    for idx in range(len(text_list)):
        if (text_list[idx][0]=='Ġ'):
            text_list[idx]=text_list[idx][1:]
            string+=" "
        if (text_list[idx]=="</"):
            text_list[idx]="["
        if (text_list[idx]==">"):
            text_list[idx]="]"
        if (attention_list[idx]<0):
            string += "\\colorbox{%s!%s}{"%('red', -attention_list[idx])+"\\strut " + text_list[idx]+"}"
        else:
            string += "\\colorbox{%s!%s}{"%('green', attention_list[idx])+"\\strut " + text_list[idx]+"}"
    string += "\n}}}"
    return string

def generate_heatmap(ModelWapper,sentense):
    global model, tokenizer
    model = ModelWapper.model.cuda()
    tokenizer = ModelWapper.tokenizer
    tokenized_sentense = tokenizer(sentense)
    words = tokenizer.convert_ids_to_tokens(tokenized_sentense.input_ids)
    lig = LayerIntegratedGradients(squad_pos_forward_func, model.deberta.embeddings)
    attributions, delta = lig.attribute(inputs=torch.tensor([tokenized_sentense.input_ids]).to('cuda:0'),
                                additional_forward_args=(2),
                                return_convergence_delta=True)
    return generate_HeatmapText(summarize_attributions(attributions), words)

def generate_InputReductionText(str1):
    str1=str1.replace("%", "\%")
    str1=str1.replace("-->", "to")
    str1=str1.replace("<del>s</del>", "s")
    str1=str1.replace("<<del>/s</del>>", "</s>")
    str1=str1.replace("</s>", "[SEP]")
    str1=str1.replace("</>", "[SEP]")
    str1=str1.replace("<del>", "\sout{")
    str1=str1.replace("</del>", "}")
    return str1


In [ ]:
def generate_HeatmapTextElectra(attention_list, text_list):
    string = r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
    for idx in range(len(text_list)):
        if (text_list[idx][0]=='#' and text_list[idx][1]=='#'):
            text_list[idx]=text_list[idx][2:]
        else:
            string+=" "
        if (text_list[idx]=="<"):
            text_list[idx]="["
        if (text_list[idx]==">"):
            text_list[idx]="]"
        if (attention_list[idx]<0):
            string += "\\colorbox{%s!%s}{"%('red', -attention_list[idx])+"\\strut " + text_list[idx]+"}"
        else:
            string += "\\colorbox{%s!%s}{"%('green', attention_list[idx])+"\\strut " + text_list[idx]+"}"
    string += "\n}}}"
    return string


# 3. Load the trainned model

In [ ]:
electra = pipeline("text-classification", model='/content/drive/MyDrive/outputs/fewshoteletra')

deberta = pipeline("text-classification", model='/content/drive/MyDrive/outputs/fewshotdeberta')

In [ ]:
bart = pipeline("text-classification", model='facebook/bart-large-mnli')

roberta = pipeline("text-classification", model='roberta-large-mnli')

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
bart_list, roberta_list = [], []

In [ ]:
def generate_heatmap_deberta(ModelWapper,sentense):
    global model, tokenizer
    model = ModelWapper.model.cuda()
    tokenizer = ModelWapper.tokenizer
    tokenized_sentense = tokenizer(sentense)
    words = tokenizer.convert_ids_to_tokens(tokenized_sentense.input_ids)
    lig = LayerIntegratedGradients(squad_pos_forward_func, model.deberta.embeddings)
    attributions, delta = lig.attribute(inputs=torch.tensor([tokenized_sentense.input_ids]).to('cuda:0'),
                                additional_forward_args=(2),
                                return_convergence_delta=True)
    return generate_HeatmapText(summarize_attributions(attributions), words)

def generate_heatmap_Eletra(ModelWapper,sentense):
    global model, tokenizer
    model = ModelWapper.model.cuda()
    tokenizer = ModelWapper.tokenizer
    tokenized_sentense = tokenizer(sentense)
    words = tokenizer.convert_ids_to_tokens(tokenized_sentense.input_ids)
    lig = LayerIntegratedGradients(squad_pos_forward_func, model.electra.embeddings)
    attributions, delta = lig.attribute(inputs=torch.tensor([tokenized_sentense.input_ids]).to('cuda:0'),
                                additional_forward_args=(2),
                                return_convergence_delta=True)
    return generate_HeatmapTextElectra(summarize_attributions(attributions), words)

# 4. Use captum to generate the heatmap

In [ ]:
def generate_heatmap_bart(ModelWapper,sentense):
    global model, tokenizer
    model = ModelWapper.model.cuda()
    tokenizer = ModelWapper.tokenizer
    tokenized_sentense = tokenizer(sentense)
    words = tokenizer.convert_ids_to_tokens(tokenized_sentense.input_ids)
    lig = LayerIntegratedGradients(squad_pos_forward_func, model.model.decoder.embed_tokens)
    attributions, delta = lig.attribute(inputs=torch.tensor([tokenized_sentense.input_ids]).to('cuda:0'),
                                additional_forward_args=(2),
                                return_convergence_delta=True)
    return generate_HeatmapText(summarize_attributions(attributions), words)

def generate_heatmap_roberta(ModelWapper,sentense):
    global model, tokenizer
    model = ModelWapper.model.cuda()
    tokenizer = ModelWapper.tokenizer
    tokenized_sentense = tokenizer(sentense)
    words = tokenizer.convert_ids_to_tokens(tokenized_sentense.input_ids)
    lig = LayerIntegratedGradients(squad_pos_forward_func, model.roberta.embeddings)
    attributions, delta = lig.attribute(inputs=torch.tensor([tokenized_sentense.input_ids]).to('cuda:0'),
                                additional_forward_args=(2),
                                return_convergence_delta=True)
    return generate_HeatmapText(summarize_attributions(attributions), words)


In [ ]:
roberta.model.cuda()
generate_heatmap_roberta(roberta, text[2])

In [ ]:
ans = 0
text=[]

# classes = ["ad populum", "ad hominem", "hoc ergo propter hoc", "circular", "straw man"]
classes = ["ad populum"]
for i in custom_dataset:
    flag = False
    for j in classes:
        flag = flag or (j in i[0])
    if flag and len(i[0].split(' '))<100:
        ans+=1
        text.append(i[0])
print(ans)    

28


In [ ]:
len(text)

78

In [ ]:
text[2]

'the text has the logical fallacy of ad populum </s> sm 3: polls concerning global warming'

In [ ]:
electra.model.cuda()
generate_heatmap_Eletra(electra, text[5])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


'{\\setlength{\\fboxsep}{0pt}\\colorbox{white!0}{\\parbox{0.9\\textwidth}{\n \\colorbox{green!0.0}{\\strut [CLS]} \\colorbox{red!3.1542557408872947}{\\strut the} \\colorbox{green!6.692988589686512}{\\strut text} \\colorbox{red!35.590014796744384}{\\strut has} \\colorbox{red!11.656972042887865}{\\strut the} \\colorbox{red!1.5785514568958412}{\\strut logical} \\colorbox{green!10.769713116954025}{\\strut fall}\\colorbox{green!1.4167160444689817}{\\strut acy} \\colorbox{green!13.880719136473251}{\\strut of} \\colorbox{green!34.51811758917833}{\\strut ad} \\colorbox{green!100.0}{\\strut pop}\\colorbox{red!36.065521556793534}{\\strut ulum} \\colorbox{red!51.11871353451354}{\\strut [} \\colorbox{red!5.812777827974576}{\\strut /} \\colorbox{green!16.82247363496953}{\\strut s} \\colorbox{green!22.26415357981905}{\\strut ]} \\colorbox{green!63.60910316302101}{\\strut "} \\colorbox{green!47.85822915866994}{\\strut since} \\colorbox{green!22.453522049347168}{\\strut 88} \\colorbox{green!11.4742843

In [ ]:
str1=""
for i in text[0:10]:
    str1+=generate_heatmap_Eletra(electra, i)+"\n\n\n"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [ ]:
str1

'{\\setlength{\\fboxsep}{0pt}\\colorbox{white!0}{\\parbox{0.9\\textwidth}{\n \\colorbox{green!0.0}{\\strut [CLS]} \\colorbox{red!20.22905163809291}{\\strut the} \\colorbox{red!37.59279437333412}{\\strut text} \\colorbox{red!25.23846308576987}{\\strut has} \\colorbox{red!41.40167506026359}{\\strut the} \\colorbox{red!30.118227081460837}{\\strut logical} \\colorbox{red!11.723237509331785}{\\strut fall}\\colorbox{red!20.000211147432793}{\\strut acy} \\colorbox{red!35.95563060486528}{\\strut of} \\colorbox{red!13.294312692499002}{\\strut ad} \\colorbox{red!36.61576742577439}{\\strut pop}\\colorbox{green!80.64915684019888}{\\strut ulum} \\colorbox{red!86.67287461436773}{\\strut [} \\colorbox{red!51.33937355631515}{\\strut /} \\colorbox{red!3.19766543123391}{\\strut s} \\colorbox{green!22.90595460907595}{\\strut ]} \\colorbox{red!9.142285987810881}{\\strut what} \\colorbox{red!24.575138455881333}{\\strut type} \\colorbox{green!98.39400304553108}{\\strut of} \\colorbox{green!35.6580297442049}

In [ ]:
deberta.model.cuda()
deberta_list = []
for i in text:
    deberta_list.append(generate_heatmap_deberta(deberta, i))

In [ ]:
deberta_list[6]

'{\\setlength{\\fboxsep}{0pt}\\colorbox{white!0}{\\parbox{0.9\\textwidth}{\n\\colorbox{green!0.0}{\\strut [CLS]}\\colorbox{red!2.9161050294768356}{\\strut the} \\colorbox{red!3.487893498052339}{\\strut text} \\colorbox{red!26.633145521482227}{\\strut has} \\colorbox{red!10.644462214367238}{\\strut the} \\colorbox{red!5.776591826403865}{\\strut logical} \\colorbox{red!8.1826786306368}{\\strut fallacy} \\colorbox{red!47.808092736202745}{\\strut of} \\colorbox{red!55.16144182287945}{\\strut ad} \\colorbox{red!100.0}{\\strut pop}\\colorbox{red!16.299467889239754}{\\strut ulum} \\colorbox{red!29.85223287994327}{\\strut [}\\colorbox{green!9.980629330444847}{\\strut s}\\colorbox{green!4.564821830830524}{\\strut ]} \\colorbox{green!9.944848355251064}{\\strut the} \\colorbox{red!7.126842192843305}{\\strut phrase} \\colorbox{green!18.555945292253092}{\\strut "}\\colorbox{green!3.0089248021635724}{\\strut nin}\\colorbox{green!24.556619234948656}{\\strut ety} \\colorbox{green!26.81786878101145}{\\

In [ ]:
electra_list_total = []
deberta_list_total = []
electra.model.cuda()
for i in custom_dataset:
    electra_list_total.append(generate_heatmap_Eletra(electra, i[0]))
deberta.model.cuda()
for i in custom_dataset:
    deberta_list_total.append(generate_heatmap_deberta(deberta, i[0]))

In [ ]:
import pandas as pd
import numpy as np
deberta_InputReduction = []
electra_InputReduction = []
datas = np.array(pd.read_csv('inputReductionDeberta.csv'))
deberta_InputReduction = datas[:,2]
datas = np.array(pd.read_csv('inputReductionEletra.csv'))
electra_InputReduction = datas[:,2]

In [ ]:
generate_InputReductionText(electra_InputReduction[0])

'\\sout{2 (100\\%)} to \\sout{2 (99\\%)}\n\n\\sout{the text has the} logical fallacy \\sout{of cum hoc} ergo propter \\sout{hoc} [SEP] \\sout{since} lucy \\sout{sutton} became vice \\sout{president} of the \\sout{parent-teacher} association, student performance has \\sout{declined and} teacher \\sout{morale is} down. we \\sout{on the school} board \\sout{believe} that sutton \\sout{bears sole} responsibility \\sout{for the downtrend}.\n\nlogical fallacy ergo propter [SEP] lucy became vice of the association, student performance has teacher down. we board that sutton responsibility.'

In [ ]:
def localize_sentence(sentence):
    for i in range(len(custom_dataset)):
        if (custom_dataset[i][0]==sentence):
            return i

In [ ]:
string = r'''\documentclass{article}
\usepackage[utf8]{inputenc}

\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\usepackage{ulem}
\usepackage{verbatim}

\begin{document}

'''
for i in range(len(text)):
    locate = localize_sentence(text[i])
    if (("SKIPPED" in deberta_InputReduction[locate]) or ("SKIPPED" in electra_InputReduction[locate])):
        continue
    string += "\section{}\n"
    string += generate_InputReductionText(text[i])

    string += "\subsection{deberta}\n"
    string += "\paragraph{Heatmap}\n"
    string += deberta_list[i]+"\n\n"
    string += "\paragraph{InputReduction}\n"
    string += generate_InputReductionText(deberta_InputReduction[locate])+"\n\n"

    string += "\subsection{electra}\n"
    string += "\paragraph{Heatmap}\n"
    string += electra_list[i]+"\n\n"
    string += "\paragraph{InputReduction}\n"
    string += generate_InputReductionText(electra_InputReduction[locate])+"\n\n"

    


In [ ]:
with open("out.tex","w") as f:
    f.write(string+"\end{document}"+"\n")